Let's just try to get this to work.

I need to read in the coordinates of the slab and the oleic ion file

Identify sites on the slab, create a system with the slab and add the oleic ligands to the sites

then I need to output the...



What I could do is try to do the slabs and ligands separately in mbuild and then convert them to separate BGFs and then reparameterize them based on the original structure files and then combine and then convert to LAMMPS input

In [11]:
import mbuild as mb
from mbuild.lib.recipes import TiledCompound
import parmed as pmd
import numpy as np
import openmm
import warnings
from random import sample
from copy import deepcopy
import openbabel as ob
from copy import deepcopy
# from slab_builder import *
warnings.simplefilter("ignore")

In [12]:
slab = mb.load("large_slab.mol2")

In [13]:
oleic_ion = mb.load("test_oleic_ion.mol2")
# oleic_ion.save("ok_oleic.mol2")

In [14]:
oleic_acid = mb.load("oleic.gaff.mol2")

In [15]:
slab.visualize(backend="nglview")

NGLWidget()

In [25]:
small_slab = mb.load('../ZnS_small_slab.mol2')
small_slab.visualize(backend="nglview")

NGLWidget()

In [26]:
oleic_ion.visualize(backend="nglview")

NGLWidget()

In [27]:
oleic_acid.visualize(backend="nglview")

NGLWidget()

In [22]:
for S in slab:
    if S.element.symbol == 'S' and S.pos[2] > 1.3:
        print(S.pos[2])


1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108718872
1.3373500108

In [20]:
for S in slab:
    if S.element.symbol == 'S' and S.pos[2] > 1.3:
        # print(S)
        port = mb.Port(anchor=S, orientation=[0,0,1], separation=0.05)
        slab.add(port)

In [23]:
slab.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
from scipy.spatial.distance import pdist, squareform
import random


def select_coordinates(ports, num_to_select, min_distance):
    
    coords = [port.pos for port in ports]

    num_coords = len(coords)
    if num_to_select > num_coords:
        print("Warning: Not enough coordinates available to select.")
        return None
    
    distance_matrix = squareform(pdist(coords))

    valid_indices = set(range(num_coords))
    selected_ports = []

    for _ in range(num_to_select):
        if not valid_indices:
            print("Warning: Unable to find enough coordinates with the given constraints.")
            break

        index = random.choice(list(valid_indices))
        selected_ports.append(ports[index])
        valid_indices.remove(index)

        too_close_indices = set(np.where(distance_matrix[index] < min_distance)[0])
        valid_indices -= too_close_indices

    return selected_ports

In [8]:
# Attempt to place a ligand on the slab

def place_ligand(ligand_file, port, slab, slab_coords):
    
    # 1) Generate the ligand structure
    ligand = mb.load(ligand_file)

    # 2) Translate the ligand to the slab coordinate
    ligand.add(mb.Port(anchor=ligand[0], separation=0.12, orientation=(ligand[2].pos - ligand[5].pos)))
    
    bond_orientation = port.pos - port.anchor.pos
    bond_orientation = bond_orientation / np.linalg.norm(bond_orientation)
    
    mb.force_overlap(move_this=ligand,
                     from_positions=ligand['Port[0]'],
                     to_positions=port)
    
    slab.add(ligand, label="OLC[$]")
    
    # print(bond_orientation, orient)

    coords = np.array(slab_coords)  # Convert list to numpy array for easier manipulation
    # print(slab_coords)
    min_coords = np.min(slab_coords, axis=0)
    max_coords = np.max(slab_coords, axis=0)
    # dimensions = max_coords - min_coords

    # 3) Check if ligand's x,y coordinates are within bounds of slab
    for attempt in range(36):  # Rotate up to 360 degrees
        x_coords = ligand.xyz[:, 0]
        y_coords = ligand.xyz[:, 1]

        min_x, max_x = np.min(x_coords), np.max(x_coords)
        min_y, max_y = np.min(y_coords), np.max(y_coords)
        
        if (min_x >= min_coords[0] and max_x <= max_coords[0] and
            min_y >= min_coords[1] and max_y <= max_coords[1]):
            # 3a) If they are within the bounds, return True
            
            
            return True, ligand
        else:
            # 3b) If they are not, try rotating by 10 degrees
            ligand.translate(-port.pos)
            ligand.rotate(theta=np.deg2rad(10), around=bond_orientation)  # Rotate around z-axis
            ligand.translate(port.pos)
            # slab.remove(ligand)

    # If no suitable orientation found after rotating through all 360 degrees, return False
    slab.remove(ligand)
    return False, ligand

In [20]:
for i,atom in enumerate(oleic_ion):
    print(i, atom)

0 <O pos=([ 0.0978 -0.424   0.8059]), 1 bonds, id: 139952542028880>
1 <O pos=([-0.0341 -0.3933  0.6313]), 1 bonds, id: 139952550452944>
2 <C pos=([ 0.4341 -0.3823  0.1447]), 4 bonds, id: 139952550451920>
3 <C pos=([ 0.4145 -0.4006 -0.0073]), 4 bonds, id: 139952550016848>
4 <C pos=([ 0.301  -0.4008  0.2205]), 4 bonds, id: 139952550454480>
5 <C pos=([ 0.5479 -0.3841 -0.0835]), 4 bonds, id: 139952948122768>
6 <C pos=([ 0.3201 -0.3836  0.3726]), 4 bonds, id: 139952535778256>
7 <C pos=([ 0.8266 -0.1797 -0.5636]), 4 bonds, id: 139952550017168>
8 <C pos=([ 0.8747 -0.2362 -0.6992]), 4 bonds, id: 139952550016400>
9 <C pos=([ 0.6798 -0.2188 -0.5363]), 4 bonds, id: 139952541782288>
10 <C pos=([ 0.5323 -0.4097 -0.235 ]), 4 bonds, id: 139952550455568>
11 <C pos=([ 0.187  -0.4035  0.4482]), 4 bonds, id: 139952550451024>
12 <C pos=([ 1.0178 -0.1909 -0.7361]), 4 bonds, id: 139952550448400>
13 <C pos=([ 0.631  -0.1638 -0.4008]), 4 bonds, id: 139952542168528>
14 <C pos=([ 0.4447 -0.3055 -0.3022]), 3 bon

In [18]:
mb.clone?

Signature: mb.clone(existing_compound, clone_of=None, root_container=None)
Docstring:
Clone Compound.

A faster alternative to deepcopying. Does not resolve circular dependencies.
This should be safe provided you never try to add the top of a Compound
hierarchy to a sub-Compound.

Parameters
----------
existing_compound : mbuild.Compound
    Existing Compound that will be copied
clone_of : dict, optional, default None,
root_container : mb.Compound, optional, default None,
File:      ~/miniconda3/envs/dev/lib/python3.11/site-packages/mbuild/compound.py
Type:      function

In [37]:
test = mb.clone(oleic_ion)
test.add(mb.Port(anchor=test[0], separation=0.12, orientation=(test[2].pos - test[5].pos)))

In [38]:
test.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [28]:
test.remove(test[2])
test.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [30]:
test.remove(test[3])
test.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [32]:
test.remove(test[5])
test.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Position charged ligands on Zinc atoms

In [24]:
test_slab = mb.load("large_slab.mol2")
for Zn in test_slab:
    if Zn.element.symbol == 'Zn' and Zn.pos[2] > 1:
        port = mb.Port(anchor=Zn, orientation=[0,0,1], separation=0.05)
        test_slab.add(port)

ports = test_slab.all_ports()

coords = [port.pos for port in test_slab.all_ports()]

# Randomly select N sites from `coords`, with a minimum of X distance between them
# select_coordinates(coords, N, X)
# will throw a warning if unable to find specified N sites given coords + X criteria
selected_sites = select_coordinates(test_slab.all_ports(), 26, 0.3)

coord = ports[4]  # replace with your actual coordinate
slab_coords = [port.pos for port in test_slab.all_ports()]  # replace with your actual slab dimensions

ligands = []
missing_index = []
for i,site in enumerate(selected_sites):
    result, ligand = place_ligand("test_oleic_ion.mol2", site, test_slab, slab_coords)
    if result == False:
        print(f"missed this one: {i}")
        missing_index.append(i)
    else:
        ligands.append(ligand)

missed this one: 4


ValueError: There is no edge between <O pos=([3.2263 0.1927 1.4287]), 0 bonds, id: 140355784701120> and <C pos=([3.1519 0.2133 1.534 ]), 0 bonds, id: 140355784715472>

In [105]:
test_slab.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Remove bonds between oleic acid ions and Zn atoms

In [115]:
zn_bond_list = []
for bond in test_slab.bonds():
    if bond[0].name == 'ZN' or bond[1].name == 'ZN':
        zn_bond_list.append(bond)

In [116]:
for bond in zn_bond_list:
    test_slab.remove_bond(bond)

In [117]:
test_slab.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Check and adjust charges

In [46]:
for Zn in test_slab:
    if Zn.element.symbol == 'Zn' and Zn.pos[2] > 1:
        print(Zn.charge)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [70]:
for i in test_slab['OLC[0]']:
   print(i.charge)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [127]:
ligand_system = deepcopy(test_slab)

TypeError: object of type 'generator' has no len()

In [132]:
slab_atoms = []
for atom in ligand_system:
    if atom.name == "ZN" or atom.name == "S":
        slab_atoms.append(atom)

for at in slab_atoms:
    ligand_system.remove(at)

In [133]:
ligand_system.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [134]:
ligand_system.save("ligand_system.mol2")

In [92]:
lig_system = mb.Compound()
for i in ligands:
    lig = mb.load('test_oleic_ion.mol2')
    lig.translate(i.pos)
    lig_system.add(lig)

In [94]:
lig_system.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [114]:
test_slab.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [106]:
for i in ligands:
    print(i)

<Compound 53 particles, 52 bonds, non-periodic, id: 139709881522384>
<Compound 53 particles, 52 bonds, non-periodic, id: 139710486533136>
<Compound 53 particles, 52 bonds, non-periodic, id: 139710476421648>
<Compound 53 particles, 52 bonds, non-periodic, id: 139710750200464>
<Compound 53 particles, 52 bonds, non-periodic, id: 139710487383504>
<Compound 53 particles, 52 bonds, non-periodic, id: 139709719714320>
<Compound 53 particles, 52 bonds, non-periodic, id: 139709786513616>
<Compound 53 particles, 52 bonds, non-periodic, id: 139709885369360>
<Compound 53 particles, 52 bonds, non-periodic, id: 139710973257552>
<Compound 53 particles, 52 bonds, non-periodic, id: 139709912840912>
<Compound 53 particles, 52 bonds, non-periodic, id: 139710488577040>
<Compound 53 particles, 52 bonds, non-periodic, id: 139709902852048>
<Compound 53 particles, 52 bonds, non-periodic, id: 139709907715152>
<Compound 53 particles, 52 bonds, non-periodic, id: 139709907798992>
<Compound 53 particles, 52 bonds, 

## Just fucking brute force copy the position for every ligand and write the file

In [138]:
ligands[0]

<Compound 53 particles, 52 bonds, non-periodic, id: 139709881522384>

In [143]:
test_oleic_clone = mb.load("test_oleic_ion.mol2")
test_oleic_clone.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [141]:
for i,particle in enumerate(ligands[0].particles()):
    test_oleic_clone[i].translate_to(particle.pos)

In [142]:
test_oleic_clone.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [147]:
oleic_charges = []
with open("oleic_charges.txt") as f:
    for line in f:
        element = line.strip().split()[0]
        charge = float(line.strip().split()[1])
        oleic_charges.append((element, charge))

In [148]:
oleic_charges

[('O', -0.8538),
 ('O1', -0.8538),
 ('C', -0.0794),
 ('C1', -0.0804),
 ('C2', -0.0784),
 ('C3', -0.0764),
 ('C4', -0.0784),
 ('C5', -0.0794),
 ('C6', -0.0784),
 ('C7', -0.0774),
 ('C8', -0.0502),
 ('C9', -0.0654),
 ('C10', -0.0774),
 ('C11', -0.0452),
 ('C12', -0.1592),
 ('C13', -0.0814),
 ('C14', -0.1902),
 ('C15', -0.1944),
 ('C16', -0.0921),
 ('C17', 0.9076),
 ('H', 0.0342),
 ('H1', 0.0342),
 ('H2', 0.0387),
 ('H3', 0.0387),
 ('H4', 0.0367),
 ('H5', 0.0367),
 ('H6', 0.0382),
 ('H7', 0.0382),
 ('H8', 0.0257),
 ('H9', 0.0257),
 ('H10', 0.0392),
 ('H11', 0.0392),
 ('H12', 0.0387),
 ('H13', 0.0387),
 ('H14', 0.0402),
 ('H15', 0.0402),
 ('H16', 0.0437),
 ('H17', 0.0437),
 ('H18', 0.0437),
 ('H19', 0.0437),
 ('H20', 0.0372),
 ('H21', 0.0372),
 ('H22', 0.0452),
 ('H23', 0.0452),
 ('H24', 0.128),
 ('H25', 0.0407),
 ('H26', 0.0407),
 ('H27', 0.119),
 ('H28', 0.0187),
 ('H29', 0.0187),
 ('H30', 0.031033),
 ('H31', 0.031033),
 ('H32', 0.031033)]

In [149]:
test_oleic_clone = mb.load("test_oleic_ion.mol2")
for i,at in enumerate(test_oleic_clone):
    if at.element.symbol in oleic_charges[i][0]:
        at.charge = oleic_charges[i][1]

In [150]:
for i,ligand in enumerate(ligands):
    for j,particle in enumerate(ligand.particles()):
        test_oleic_clone[j].translate_to(particle.pos)
    test_oleic_clone.save(f"ligand_files/ligand_{i}.mol2", overwrite=True)

In [151]:
zns = []
for at in test_slab:
    if at.element.symbol == "Zn":
        zns.append(at)
        
len(zns)

500

In [153]:
high_zn = []
test_slab = mb.load("large_slab.mol2")
for at in test_slab:
    if at.element.symbol == 'Zn':
        at.charge = 2
        if at.pos[2] > 1:
            at.charge += 17.0342/100
    elif at.element.symbol == "S":
        at.charge = -2

In [154]:
test_slab.save("test_slab_charged.mol2")

In [9]:
from pymatgen.ext.matproj import MPRester
from pymatgen.core.surface import SlabGenerator
from pymatgen.core.surface import generate_all_slabs

# Replace 'API_KEY' with your Materials Project API Key
mpr = MPRester("Ry4dD0RphvMG7eTh15jWufw7xLXt8J6b")

# Get a structure
structure = mpr.get_structure_by_material_id("mp-10695")  # Replace "mp-1234" with the id of the structure you are interested in.

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
slab_gen = SlabGenerator(structure, (1,1,0), 10, 10)
slab = slab_gen.get_slabs()[0]

In [11]:
from pymatgen.transformations.standard_transformations import SupercellTransformation

transformation = SupercellTransformation([[2,0,0],[0,2,0],[0,0,2]])
larger_slab = transformation.apply_transformation(slab)

In [12]:
from pymatgen.vis.plotly_plots import PlotlyStructure

plotter = PlotlyStructure(larger_slab)
plotter.show()

ModuleNotFoundError: No module named 'pymatgen.vis.plotly_plots'

In [20]:
larger_slab.to(filename="larger_slab", fmt="poscar")